# Get data from Renewable ninja API

First we try a method to get data from random coordinates in a country of interest and then aggregate them to have a single time series as an output.

In [39]:
import requests
import pandas as pd
import random
from opencage.geocoder import OpenCageGeocode
import io

# Your Renewable Ninja API token
api_token = 'e2be88b4f8ae85f401fdf6b205165098e3cf4e37'

opencage_api_key ='b42473080f3a4bdb9033973e58fea766'
# Define the endpoint
base_url = ' https://www.renewables.ninja/api/data/'

In [56]:
# Function to get random coordinates within a country
def get_random_coordinates(country, num_locations):
    geocoder = OpenCageGeocode(opencage_api_key)
    result = geocoder.geocode(country, no_annotations='1')
    
    if not result:
        raise ValueError(f"Cannot find location for the country: {country}")
    
    first_result = result[0]
    # print(first_result)
    # if 'geometry' not in first_result or 'bounds' not in first_result['geometry']:
    #     raise ValueError(f"Cannot find bounding box information for {country}")
    
    bbox = first_result['bounds']
    min_lat, max_lat = bbox['southwest']['lat'], bbox['northeast']['lat']
    min_lon, max_lon = bbox['southwest']['lng'], bbox['northeast']['lng']
    
    coordinates = []
    for _ in range(num_locations):
        lat = random.uniform(min_lat, max_lat)
        lon = random.uniform(min_lon, max_lon)
        coordinates.append((lat, lon))
    return coordinates

In [57]:
country = 'United Kingdom'
num_locations = 5
get_random_coordinates(country, num_locations)

{'bounds': {'northeast': {'lat': 61.061, 'lng': 2.0919117}, 'southwest': {'lat': 49.674, 'lng': -14.015517}}, 'components': {'ISO_3166-1_alpha-2': 'GB', 'ISO_3166-1_alpha-3': 'GBR', '_category': 'place', '_type': 'country', 'continent': 'Europe', 'country': 'United Kingdom', 'country_code': 'gb'}, 'confidence': 1, 'formatted': 'United Kingdom', 'geometry': {'lat': 54.7023545, 'lng': -3.2765753}}


[(49.89625410266045, -11.178399966843148),
 (58.52114628331448, -2.0577601920032507),
 (60.94440961143393, -11.468384502421454),
 (55.25122077262681, -12.652702624749848),
 (60.681296068427194, -6.5252407765482685)]

In [67]:
def get_renewable_data(lat, lon, technology, api_token, year=2020):
    headers = {
        'Authorization': 'Token ' + api_token,
    }
    params = {
        'lat': lat,
        'lon': lon,
        'date_from': f'{year}-01-01',
        'date_to': f'{year}-12-31',
        'dataset': 'merra2',
        'capacity': 1,
        'format': 'csv'
    }
    
    if technology == 'pv':
        params.update({
            'system_loss': 0.1,
            'tracking': 0,
            'tilt': 35,
            'azim': 180
        })
    elif technology == 'wind':
        params.update({
            'height': 100,  # Specify an appropriate height
            'turbine': 'Vestas V80 2000'  # Specify a turbine model
        })
    
    url = f"{base_url}{technology}"
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        # Skip potential metadata and read the CSV data correctly
        csv_data = response.text.split('\n', 1)[1]
        return pd.read_csv(io.StringIO(csv_data), skiprows=2, index_col=0)
    else:
        raise Exception(f"Error {response.status_code}: {response.text}")

In [69]:
lat = 45
lon = 10
tech = 'wind'
year = '2021'
get_renewable_data(lat, lon, tech, api_token, year)

,electricity
time,
2021-01-01 00:00,0.069
2021-01-01 01:00,0.091
2021-01-01 02:00,0.102
2021-01-01 03:00,0.100
2021-01-01 04:00,0.102
...,...
2021-12-31 19:00,0.340
2021-12-31 20:00,0.402
2021-12-31 21:00,0.465


In [70]:
# Function to fetch data for multiple random locations and average the results
def fetch_and_average_data(country, num_locations, technologies, api_token, year=2020):
    coordinates = get_random_coordinates(country, num_locations)
    all_data = {}
    
    for tech in technologies:
        tech_data = []
        for lat, lon in coordinates:
            try:
                data = get_renewable_data(lat, lon, tech, api_token, year)
                tech_data.append(data)
            except Exception as e:
                print(f"Error fetching data for location ({lat}, {lon}) with technology {tech}: {e}")
        
        if tech_data:
            combined_df = pd.concat(tech_data).groupby('time').mean().reset_index()
            all_data[tech] = combined_df
    
    return all_data

In [71]:

# Specify parameters
country = 'United Kingdom'
num_locations = 5
technologies = ['pv', 'wind']
year = 2020

# Fetch and average data
averaged_data = fetch_and_average_data(country, num_locations, technologies, api_token, year)

# Save averaged data to CSV files
# for tech, df in averaged_data.items():
#     df.to_csv(f'averaged_{tech}_data_{country}.csv', index=False)

print("Data retrieval, averaging, and saving complete.")

{'bounds': {'northeast': {'lat': 61.061, 'lng': 2.0919117}, 'southwest': {'lat': 49.674, 'lng': -14.015517}}, 'components': {'ISO_3166-1_alpha-2': 'GB', 'ISO_3166-1_alpha-3': 'GBR', '_category': 'place', '_type': 'country', 'continent': 'Europe', 'country': 'United Kingdom', 'country_code': 'gb'}, 'confidence': 1, 'formatted': 'United Kingdom', 'geometry': {'lat': 54.7023545, 'lng': -3.2765753}}
Error fetching data for location (51.01600800236232, -7.030332635134643) with technology wind: Error 429: Error: Reached limit (50/hour) Expected available in 2001 seconds.
Error fetching data for location (58.8341406316656, -8.507913210670575) with technology wind: Error 429: Error: Reached limit (50/hour) Expected available in 2000 seconds.
Error fetching data for location (49.96171263325312, -1.726834881494165) with technology wind: Error 429: Error: Reached limit (50/hour) Expected available in 1999 seconds.
Error fetching data for location (57.03450732673516, -6.6426205312321285) with tech

In [14]:
averaged_data

{}